In [1]:
from functions import *

import datashader as ds
import datashader.transfer_functions as tf
import xarray as xr
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html

%load_ext autoreload
%autoreload 2

/tmp/ipykernel_19829/4239568921.py:10: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_19829/4239568921.py:11: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
DATA_PATH = '/home/helvecioneto/disco1/dados/goes16/mmh/'
DATA_NAME = 'goes16'
DATA_PATTERN = '%%YYYY%%mm%%dd_%%HH%%MM.nc'
TRACK_PATH = '/home/helvecioneto/disco1/tracks/goes16mmh/track/'
FORECAST_PATH = '/home/helvecioneto/disco1/tracks/goes16mmh/forecast/'
TIME_DELTA = '10min'
LAT_MIN = -56
LAT_MAX = 35
LON_MIN = -116
LON_MAX = -25

In [3]:
# PATHS
timestamp = get_latest_timestamp(TRACK_PATH)
pth_trck_bound, pth_trck_vec, pth_trck_trj = get_geotrack_paths(TRACK_PATH, timestamp)
pth_fcst_bound, pth_fcst_vec, pth_fcst_trj = get_geofrcst_paths(FORECAST_PATH, timestamp, TIME_DELTA, 1)
data_path = get_data_paths(timestamp, DATA_PATH+timestamp.strftime('%Y/%m/%d/'), DATA_PATTERN)

In [4]:
# READ
data = read_data(DATA_NAME)(data_path) * 100
data[data == 0] = np.nan
trck_bound = read_geoframe(pth_trck_bound)
trck_vec = read_geoframe(pth_trck_vec)
trck_trj = read_geoframe(pth_trck_trj)
fcst_bound = read_geoframe(pth_fcst_bound)
fcst_vec = read_geoframe(pth_fcst_vec)
fcst_trj = read_geoframe(pth_fcst_trj)


In [5]:
cvs = ds.Canvas(plot_width=data.shape[0], plot_height=data.shape[1])
array = xr.DataArray(data, dims=['y', 'x'])
img = tf.shade(cvs.raster(array), cmap=px.colors.sequential.Viridis, how='eq_hist')[::-1].to_pil()
lon_it = 0
lat_it = 0
coords = [[LON_MIN+lon_it, LAT_MIN + lat_it], [LON_MAX+lon_it, LAT_MIN +lat_it], [LON_MAX+lon_it, LAT_MAX+lat_it], [LON_MIN+lon_it, LAT_MAX+lat_it]]

In [6]:
# Read data_path using xarray
dset = xr.open_dataset(data_path)
df = dset.to_dataframe()
df = df.dropna().reset_index()

In [7]:
## Create a figure with plotly.graph_objects
fig = go.Figure()

# fig = px.scatter_mapbox(df, lat='lat', lon='lon',opacity=1,color='Band1')

# Add go scattermapbox trace
fig.add_trace(go.Scattermapbox(lat=df['lat'],lon=df['lon'], mode='markers', marker=go.scattermapbox.Marker(size=5, color=df['Band1'], colorscale='Viridis', showscale=True), text=df['Band1'], name='Band1'))


fig.update_layout(
                 mapbox_layers = [
                {
                    "sourcetype": "image",
                    "source": img,
                    "coordinates": coords,
                    "below": "traces",
                    "opacity": 1,
                }]
)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_center_lon=-50, mapbox_center_lat=-15, mapbox_zoom=3)
# Set height and width of figure to 100% of browser window
fig.update_layout(height=800, width=800, margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

# Run server with Dash
# app = dash.Dash()
# app.layout = html.Div([dcc.Graph(figure=fig)])
# app.run_server(debug=True, use_reloader=False, port=8082)